In [246]:
import pandas as pd
train = pd.read_csv('dataset/part2/ch4/train.csv')
test = pd.read_csv('dataset/part2/ch4/test.csv')

- 머신 러닝 프로세스
문제 정의 - 데이터 불러오기 - eda(탐색적 데이터 분석) - 데이터 전처리 - 검증 데이터 나누기 - 모델 학습 및 평가 - 예측 및 제출
- 데이터 전처리
concat X_train with X_validate - encoding - split

In [247]:
y_train = train.pop('Item_Outlet_Sales')
train.shape, y_train.shape, test.shape

((6818, 11), (6818,), (1705, 11))

In [248]:
train['Item_Weight'] = train['Item_Weight'].fillna(train['Item_Weight'].mean())
train['Outlet_Size'] = train['Outlet_Size'].fillna(train['Outlet_Size'].mode()[0])

test['Item_Weight'] = test['Item_Weight'].fillna(train['Item_Weight'].mean())
test['Outlet_Size'] = test['Outlet_Size'].fillna(train['Outlet_Size'].mode()[0])

In [249]:
from sklearn.preprocessing import LabelEncoder
def encode(df, cols):
    le = LabelEncoder()
    for col in cols:
        df[col] = le.fit_transform(df[col])
    return df

In [250]:
cc = pd.concat([train, test], axis=0)
cc = cc.drop('Item_Identifier', axis=1)
cols = cc.select_dtypes(include='O').columns

#todo: Why 'Outlet_Size' column filled after encoding? => Label Encoder does not allow missing values
labeled = encode(cc, cols)
train = labeled.iloc[:len(train)].copy()
test = labeled.iloc[len(train):].copy()

In [251]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(train, y_train, test_size=0.2, random_state=0)
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((5454, 10), (1364, 10), (5454,), (1364,))

In [254]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgb
from sklearn.metrics import mean_squared_error

lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_val)
lr_mse = mean_squared_error(y_val, y_pred) 

rfr = RandomForestRegressor(random_state=0)
rfr.fit(X_train, y_train)
y_pred = rfr.predict(X_val)
rfr_mse = mean_squared_error(y_val, y_pred)

lgbmr = lgb.LGBMRegressor(random_state=0, verbose=-1)
lgbmr.fit(X_train, y_train)
y_pred = lgbmr.predict(X_val)
lgbmr_mse = mean_squared_error(y_val, y_pred)

lr_mse, rfr_mse, lgbmr_mse

(1262903.8274166307, 1093374.2596630282, 1103820.8300954718)

In [257]:
pred = rfr.predict(test)
submit = pd.DataFrame({'pred': pred})
submit.to_csv('result.csv', index=False)

In [258]:
df = pd.read_csv('result.csv')
df

,pred
0,1439.612734
1,782.721138
2,2126.179036
3,1758.863834
4,2740.073268
...,...
1700,231.472028
1701,799.139766
1702,4200.146036
1703,930.229128
